In [ ]:

import qarnot
from qarnot.task import ForcedConstant, ForcedConstantAccess, ForcedNetworkRule

import os
from dotenv import load_dotenv
load_dotenv()

from qarnot.scheduling_type import OnDemandScheduling


In [ ]:
# Registry settings
ACCOUNT="miretteamin"
REPO="qarnotlab/blank-images"
TAG="qarnot_cuda-12.3.1-devel-ubuntu22.04"
PASSWORD= "dckr_pat_MuKCFvAZCZiQ1E9DRljMSqdDXG8"


# Tasq Settings
PROFILE="docker-nvidia-bi-a6000-network-ssh"
TASK_NAME=f"train all"

print(f"Creating task {TASK_NAME} with profile {PROFILE}", PASSWORD)

Creating task train all with profile docker-nvidia-bi-a6000-network-ssh dckr_pat_MuKCFvAZCZiQ1E9DRljMSqdDXG8


In [ ]:
conn = qarnot.connection.Connection(client_token='d6ef7c49b6f54cb8fd2547de73152f8149944a9bfdba34075e7e7b44c5dcdc325e12be90ffa04396fce341058b7b45cac3eb9aff786c0e0349a3425beac51d4e')

# Use qarnotlab image
task = conn.create_task(TASK_NAME, PROFILE, 1)
task.constants["DOCKER_REPO"] = REPO
task.constants["DOCKER_TAG"] = TAG
task.constants["DOCKER_SRV"] = "https://registry-1.docker.io"

task.constants["DOCKER_REGISTRY_LOGIN"] = ACCOUNT
task.constants["QARNOT_SECRET__DOCKER_REGISTRY_PASSWORD"] = PASSWORD

# Setup SSH
task.constants['DOCKER_SSH'] = 'ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMVsL2+esKcz0C2q2fsZRiPkMJjx+ECHnw+dSH6DLUyO mirette.moawad@polytechnique.edu'

# CMD

task.constants['DOCKER_CMD'] = '/bin/bash -c " \
    set_ssh && sleep infinity | tee master.logs & \
    nvidia-smi && \
    apt-get update -y && \
    apt-get install apt-utils -y && \
    apt-get install -y python3 python3-pip && \
    python3 -m pip install --upgrade pip && \
    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 && \
    pip install -r /job/code/requirements.txt && \
    python3 /job/code/run.py train --config /job/code/configs/config_memmap.json --datadir /job/ --memmap True"'


# task.resources_constraints = {
#     "cpu_count": {"min": 16}
# }


# On Demand - in order to not being preempted
# task.scheduling_type=OnDemandScheduling()

# Retrieve existing buckets
input_bucket = conn.retrieve_bucket("full.dataset")
output_bucket = conn.retrieve_bucket("output_full")

# Attach input and output buckets
task.resources.append(input_bucket)

# Assign output bucket correctly
task.results = output_bucket  

# Define checkpoints in seconds
task.snapshot(600)

# Submit the task
try:
    task.submit()
except Exception as e:
    print(f"❌ ERROR: Task submission failed: {e}")
    exit(1)  # Stop execution if submission fails


In [ ]:
import time

print("🚀 Task submitted! Waiting for execution...\n")

LAST_STATE = ''
SSH_TUNNELING_DONE = False

while not SSH_TUNNELING_DONE:
    # Fetch latest logs
    logs = task.stdout().split("\n")  # Get the last 10 lines of logs

    if task.state != LAST_STATE:
        LAST_STATE = task.state
        print(f"\n📌 Current Task State: {LAST_STATE}")

    # Print log details
    print(f"\n🔍 Checking task state: {task.state} ...")
    
    # Print logs
    print("\n📝 Task Logs:")
    for line in logs:
        print(f"   {line}")

    # Wait for the task to be FullyExecuting
    if task.state == 'FullyExecuting':
        print("\n✅ Task is now executing!")
        forward_list = task.status.running_instances_info.per_running_instance_info[0].active_forward
        if not SSH_TUNNELING_DONE and len(forward_list) != 0:
            ssh_forward_port = forward_list[0].forwarder_port
            ssh_forward_host = forward_list[0].forwarder_host
            cmd = f"ssh -o StrictHostKeyChecking=no root@{ssh_forward_host} -p {ssh_forward_port}"
            print(f"\n🛠️ SSH Command to connect: {cmd}")
            SSH_TUNNELING_DONE = True

    # Print detailed errors if task fails
    if task.state == 'Failure':
        print(f"\n❌ Task failed with errors: {task.errors}")
        SSH_TUNNELING_DONE = True

    time.sleep(10)  # Wait 10 seconds before checking again


🚀 Task submitted! Waiting for execution...


📌 Current Task State: Submitted

🔍 Checking task state: Submitted ...

📝 Task Logs:
   

📌 Current Task State: FullyDispatched

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state: FullyDispatched ...

📝 Task Logs:
   

🔍 Checking task state

In [ ]:
# task.constants['DOCKER_CMD'] = '/bin/bash -c " \
#     echo \'🔍 Ensuring SSH key exists inside the container...\' && \
#     mkdir -p /root/.ssh && chmod 700 /root/.ssh && \
#     echo \'ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMVsL2+esKcz0C2q2fsZRiPkMJjx+ECHnw+dSH6DLUyO mirette.moawad@polytechnique.edu\' > /root/.ssh/authorized_keys && \
#     chmod 600 /root/.ssh/authorized_keys && \
#     echo \'✅ SSH Key added to authorized_keys!\' && \
#     set_ssh && sleep infinity | tee master.logs & \
#     nvidia-smi && \
#     apt-get update -y && \
#     apt-get install apt-utils -y && \
#     apt-get install -y python3 python3-pip && \
#     python3 -m pip install --upgrade pip && \
#     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 && \
#     pip install -r /job/code/requirements.txt && \
#     python3 /job/code/run.py train --config /job/code/configs/config_memmap.json --datadir /job/ --memmap True"'

# task.constants['DOCKER_CMD'] = '/bin/bash -c "set_ssh && sleep infinity | tee master.logs & nvidia-smi && apt-get update -y && apt-get install apt-utils -y && apt-get install -y python3 python3-pip && python3 -m pip install --upgrade pip && pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 && pip install -r /bucket-input/full.dataset/code/requirements.txt && python3 /bucket-input/full.dataset/code/run.py train --config /bucket-input/full.dataset/code/config_memmap.json --datadir /bucket-input/full.dataset --memmap True"'
# task.constants['DOCKER_CMD'] = '/bin/bash -c " \
#     echo \'🔍 Checking all mounted filesystems:\' && df -h && \
#     echo \'🔍 Listing /mnt/:\' && ls -l /mnt/ && \
#     echo \'🔍 Listing /var/:\' && ls -l /var/ && \
#     echo \'🔍 Listing /tmp/:\' && ls -l /tmp/ && \
#     echo \'🔍 Searching for full.dataset anywhere:\' && find / -type d -name \'full.dataset\' 2>/dev/null && \
#     echo \'🔍 Searching for all requirements.txt files:\' && find / -name \'requirements.txt\' 2>/dev/null && \
#     echo \'🔍 Searching for config_memmap.json:\' && find / -name \'config_memmap.json\' 2>/dev/null && \
#     echo \'🔍 Searching for dataset files (CSV, images, etc.):\' && find / -type f \( -name \'*.csv\' -o -name \'*.jpg\' -o -name \'*.png\' \) 2>/dev/null && \
#     sleep infinity"'

# task.constants['DOCKER_CMD'] = '/bin/bash -c " \
#     echo \'🔍 Checking all mounted filesystems:\' && df -h && \
#     echo \'🔍 Listing /mnt/:\' && ls -l /mnt/ && \
#     echo \'🔍 Searching for full.dataset anywhere:\' && find / -type d -name \'full.dataset\' 2>/dev/null && \
#                 seep infinity"'

# task.constants['DOCKER_CMD'] = '/bin/bash -c " \
#     echo \'🔍 Checking all mounted filesystems:\' && df -h && \
#     echo \'🔍 Listing /cache/:\' && ls -l /cache/ && \
#     echo \'🔍 Listing /job/:\' && ls -l /job/ && \
#     echo \'🔍 Listing /share/:\' && ls -l /share/ && \
#     echo \'🔍 Searching for full.dataset in key locations:\' && find /cache/ /job/ /share/ -type d -name \'full.dataset\' 2>/dev/null && \
#     echo \'🔍 Searching for all requirements.txt files:\' && find /cache/ /job/ /share/ -name \'requirements.txt\' 2>/dev/null && \
#     echo \'🔍 Searching for config_memmap.json:\' && find /cache/ /job/ /share/ -name \'config_memmap.json\' 2>/dev/null && \
#     echo \'🔍 Searching for dataset files (CSV, images, etc.):\' && find /cache/ /job/ /share/ -type f \( -name \'*.csv\' -o -name \'*.jpg\' -o -name \'*.png\' \) 2>/dev/null"'

# task.constants['DOCKER_CMD'] = '/bin/bash -c " \
#     echo \'🔍 Checking mounted storage usage:\' && df -h || echo \'⚠️ Failed to run df -h\' && \
#     echo \'🔍 Listing /mnt/:\' && ls -lh /mnt/ || echo \'⚠️ Failed to list /mnt/\' && \
#     echo \'🔍 Listing /cache/:\' && ls -lh /cache/ || echo \'⚠️ Failed to list /cache/\' && \
#     echo \'🔍 Listing /job/:\' && ls -lh /job/ || echo \'⚠️ Failed to list /job/\' && \
#     echo \'🔍 Listing /share/:\' && ls -lh /share/ || echo \'⚠️ Failed to list /share/\' && \
#     echo \'🔍 Checking if /bucket-input/full.dataset/ exists:\' && ls -lh /bucket-input/full.dataset/ || echo \'❌ /bucket-input/full.dataset/ NOT FOUND\' && \
#     echo \'🔍 Searching for full.dataset directory (excluding /opt/):\' && find / -type d -name \'full.dataset\' ! -path "/opt/*" 2>/dev/null || echo \'⚠️ find command failed\' && \
#     echo \'🔍 Searching for large dataset files (above 1GB, excluding /opt/):\' && find / -type f -size +1G ! -path "/opt/*" 2>/dev/null || echo \'⚠️ find command failed\' && \
#     echo \'🟢 Done scanning. Keeping container alive for debugging.\' && sleep infinity"'



# task.constants['DOCKER_CMD'] = '/bin/bash -c "set_ssh && sleep infinity | tee master.logs"'
